In [1]:
using Plots
using LinearAlgebra
using Statistics
include("src/Trajectories.jl")
using .QuTaM

In [3]:
using DifferentialEquations
function rf_de!(dr, r, p, t)
    gamma = QuTaM.rf_gamma
    delta = QuTaM.rf_delta
    omega = QuTaM.rf_omega
    dr[1] = -0.5*gamma*r[1] - 2*delta*r[2]
    dr[2] = 2*delta*r[1] - 0.5*gamma*r[2] - 2*omega*r[3]
    dr[3] = 2*omega*r[2] - gamma*(r[3] + 1)
end

rf_de! (generic function with 1 method)

In [4]:
sys = QuTaM.rf_sys
params = QuTaM.rf_params

SimulParameters(psi0=ComplexF64[1.0 + 0.0im, 0.0 + 0.0im]
nsamples=75000
seed=1
ntraj=500)
multiplier=3.0
tf=25.0
dt=0.001
eps=0.001)

In [5]:
r0 = [0.0; 0.0; -1.0] # Initial Condition
tspan = (0.0, params.tf)
t_given = collect(LinRange(0, params.tf, 1000))
prob = ODEProblem(rf_de!, r0, tspan)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 25.0)
u0: 3-element Vector{Float64}:
  0.0
  0.0
 -1.0

In [6]:
sol = solve(prob, reltol = 1e-6, saveat = t_given); 

In [7]:
# Steady State
gamma = QuTaM.rf_gamma
delta = QuTaM.rf_delta
omega = QuTaM.rf_omega
#r_steady = 1/(gamma^2 + 2*omega^2+4*delta^2) * [-4*delta*omega; 2*omega*gamma;-gamma^2-4*delta^2 ]

0.5

In [ ]:
################## Average Simulation ################3
# Now from each trajectory, generate the states the given times
sample_clicks = QuTaM.run_trajectories(sys, params)
ntimes = size(t_given)[1]
sample = zeros(ComplexF64, ntimes, sys.NLEVELS, params.ntraj)
for n in 1:params.ntraj
    states = QuTaM.evaluate_at_t(t_given, sample_clicks[n], sys,  params.psi0)
    for j in 1:sys.NLEVELS
        for tn in 1:ntimes
            sample[tn, j, n] = states[tn, j]
        end 
    end 
end


In [ ]:
# Obtain the observable on the sample.
r_sample = zeros(Float64, ntimes, 3, params.ntraj)
sigma = [QuTaM.sigma_x, QuTaM.sigma_y, QuTaM.sigma_z]
for j in 1:params.ntraj
    for k in 1:3
        for tn in 1:ntimes
                r_sample[tn, k, j] = dot(sample[tn, :, j], sigma[k] * sample[tn, :, j])   # Drop the extra dimension
        end
    end
end 
# Average
r_avg = dropdims(mean(r_sample, dims=3), dims=3);

In [ ]:
tau_sample = Vector{Float64}()
for traj in sample_clicks
    if !isempty(traj)
        for click in traj
            push!(tau_sample, click.time)
        end 
    else
        continue
    end 
end 

In [2]:
QuTaM.sigma_y

2×2 Matrix{Complex{Int64}}:
 0+0im  0+1im
 0-1im  0+0im

In [ ]:
plot(sol, idxs=(0,2))
plot!(t_given, r_avg[:, 2])

In [ ]:
WTD_analytical(tau) =  (16*gamma*omega^2)*exp(-0.5*gamma*tau) * sin(0.25*tau*sqrt(16*omega^2-gamma^2))^2/(-gamma^2+16*omega^2)
histogram(tau_sample, normalize=:pdf)
plot!(t_given, WTD_analytical.(t_given), label="Analytical")